# Lab - Klasyfikacja 2

## Zadania
1. Wczytaj zbiór danych `adult.csv`. Zbadaj dane, a następnie przygotuj je do modelowania.  
   Zastosuj różne modele klasyfikacyjne do przewidywania grupy dochodów (`<=50K`/`>50K`) i wybierz odpowiednie hiperparametry. Upewnij się, że używasz odpowiednich metryk do oceny modeli. Możesz użyć zbioru walidacyjnego lub walidacji krzyżowej (zobacz: [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html), [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

In [124]:
import pandas as pd

In [125]:
adults = pd.read_csv('adult.csv', na_values='?')
adults.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [126]:
adults.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [127]:
adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [128]:
adults.isna().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

In [129]:
adults.dropna(subset=['workclass', 'occupation', 'native-country'], inplace=True)

In [130]:
adults.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K
10,65,Private,184454,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States,>50K
11,36,Federal-gov,212465,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K


In [131]:
for col in adults.columns.tolist():
    print(f"{col} - {len(adults[col].unique())}: {adults[col].unique()}")

age - 74: [25 38 28 44 34 63 24 55 65 36 26 48 43 20 37 45 22 23 54 32 46 56 17 29
 39 52 18 21 42 33 30 47 41 19 69 50 31 59 49 58 40 27 57 61 51 73 53 80
 62 35 72 64 68 66 60 67 71 70 90 77 81 74 78 82 75 85 76 89 83 79 88 87
 84 86]
workclass - 7: ['Private' 'Local-gov' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay']
fnlwgt - 26741: [226802  89814 336951 ... 199655 255835 257302]
education - 16: ['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' '5th-6th' 'Assoc-voc' '9th' 'Doctorate'
 '12th' '1st-4th' 'Preschool']
educational-num - 16: [ 7  9 12 10  6 15  4 13 14  3 11  5 16  8  2  1]
marital-status - 7: ['Never-married' 'Married-civ-spouse' 'Widowed' 'Separated' 'Divorced'
 'Married-spouse-absent' 'Married-AF-spouse']
occupation - 14: ['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' 'Other-service'
 'Prof-specialty' 'Craft-repair' 'Adm-clerical' 'Exec-managerial'
 'Tech-support' 'Sales' 'Priv-hou

In [132]:
import numpy as np

In [142]:
age_bins = [0, 18, 25, 35, 45, 55, 65, np.inf]
adults['age_group'] = pd.cut(adults['age'], bins=age_bins, labels=[0, 1, 2, 3, 4, 5, 6])

In [144]:
adults['workclass_group'] = adults['workclass'].map({
    'Private': 0,
    'Self-emp-not-inc': 1,
    'Self-emp-inc': 1,
    'Federal-gov': 2,
    'Local-gov': 2,
    'State-gov': 2,
    'Without-pay': 3,
    'Never-worked': 3
})

In [145]:
adults['fnlwgt_normalized'] = (adults['fnlwgt'] - adults['fnlwgt'].mean()) / adults['fnlwgt'].std()

In [146]:
adults['marital-status_group'] = adults['marital-status'].map({
    'Married-civ-spouse': 0,
    'Divorced': 1,
    'Never-married': 2,
    'Separated': 3,
    'Widowed': 4,
    'Married-spouse-absent': 5,
    'Married-AF-spouse': 6
})
adults['occupation_group'] = adults['occupation'].map({
    'Tech-support': 0,
    'Craft-repair': 1,
    'Other-service': 2,
    'Sales': 3,
    'Exec-managerial': 4,
    'Prof-specialty': 5,
    'Handlers-cleaners': 6,
    'Machine-op-inspct': 7,
    'Adm-clerical': 8,
    'Farming-fishing': 9,
    'Transport-moving': 10,
    'Priv-house-serv': 11,
    'Protective-serv': 12,
    'Armed-Forces': 13
})
adults['relationship_group'] = adults['relationship'].map({
    'Wife': 0,
    'Own-child': 1,
    'Husband': 2,
    'Not-in-family': 3,
    'Other-relative': 4,
    'Unmarried': 5
})
adults['race_group'] = adults['race'].map({
    'White': 0,
    'Asian-Pac-Islander': 1,
    'Amer-Indian-Eskimo': 2,
    'Other': 3,
    'Black': 4
})
adults['gender_group'] = adults['gender'].map({
    'Male': 0,
    'Female': 1
})

In [147]:
adults['capital-gain_normalized'] = (adults['capital-gain'] - adults['capital-gain'].mean()) / adults['capital-gain'].std()

In [148]:
adults['capital-gain_normalized'] = (adults['capital-gain'] - adults['capital-gain'].mean()) / adults['capital-gain'].std()
adults['capital-loss_normalized'] = (adults['capital-loss'] - adults['capital-loss'].mean()) / adults['capital-loss'].std()
adults['hours-per-week_normalized'] = (adults['hours-per-week'] - adults['hours-per-week'].mean()) / adults['hours-per-week'].std()

In [149]:
adults['income'] = adults['income'].map({
    '<=50K': 0,
    '>50K': 1
})

In [150]:
adults.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,...,workclass_group,fnlwgt_normalized,marital-status_group,occupation_group,relationship_group,race_group,gender_group,capital-gain_normalized,capital-loss_normalized,hours-per-week_normalized
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,...,0,0.350886,2,7,1,4,0,-0.146732,-0.218778,-0.078119
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,...,0,-0.945868,0,9,2,0,0,-0.146732,-0.218778,0.754693
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,...,2,1.393576,0,12,2,0,0,-0.146732,-0.218778,-0.078119
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,...,0,-0.278417,0,7,2,4,0,0.877457,-0.218778,-0.078119
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,...,0,0.084801,2,2,3,0,0,-0.146732,-0.218778,-0.910931
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,1,-0.805655,0,5,2,0,0,0.266647,-0.218778,-0.744369
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,...,0,1.703272,2,2,5,0,1,-0.146732,-0.218778,-0.078119
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,...,0,-0.802152,0,1,2,0,0,-0.146732,-0.218778,-2.576556
10,65,Private,184454,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,...,0,-0.049988,0,7,2,0,0,0.708269,-0.218778,-0.078119
11,36,Federal-gov,212465,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,White,Male,...,2,0.215169,0,8,2,0,0,-0.146732,-0.218778,-0.078119


In [151]:
adults.drop(columns=['age', 'workclass', 'fnlwgt', 'education', 'occupation', 'relationship', 'race', 'gender', 'marital-status', 'native-country', 'capital-gain', 'capital-loss', 'hours-per-week'], inplace=True)

In [152]:
adults.head(10)

,educational-num,income,age_group,workclass_group,fnlwgt_normalized,marital-status_group,occupation_group,relationship_group,race_group,gender_group,capital-gain_normalized,capital-loss_normalized,hours-per-week_normalized
0,7,0,1,0,0.350886,2,7,1,4,0,-0.146732,-0.218778,-0.078119
1,9,0,3,0,-0.945868,0,9,2,0,0,-0.146732,-0.218778,0.754693
2,12,1,2,2,1.393576,0,12,2,0,0,-0.146732,-0.218778,-0.078119
3,10,1,3,0,-0.278417,0,7,2,4,0,0.877457,-0.218778,-0.078119
5,6,0,2,0,0.084801,2,2,3,0,0,-0.146732,-0.218778,-0.910931
7,15,1,5,1,-0.805655,0,5,2,0,0,0.266647,-0.218778,-0.744369
8,10,0,1,0,1.703272,2,2,5,0,1,-0.146732,-0.218778,-0.078119
9,4,0,4,0,-0.802152,0,1,2,0,0,-0.146732,-0.218778,-2.576556
10,9,1,5,0,-0.049988,0,7,2,0,0,0.708269,-0.218778,-0.078119
11,13,0,3,2,0.215169,0,8,2,0,0,-0.146732,-0.218778,-0.078119


In [153]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [154]:
X = adults.drop(columns=['income'])
y = adults["income"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [155]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
    "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
}

In [156]:
X.isna().sum()

educational-num              0
age_group                    0
workclass_group              0
fnlwgt_normalized            0
marital-status_group         0
occupation_group             0
relationship_group           0
race_group                   0
gender_group                 0
capital-gain_normalized      0
capital-loss_normalized      0
hours-per-week_normalized    0
dtype: int64

In [159]:
from sklearn.model_selection import cross_validate

for model_name, model in models.items():
    print(f"Evaluating {model_name}")

    scores = cross_validate(model, X, y, cv=5,
                            scoring=['accuracy', 'precision', 'recall', 'f1'])

    print(f"Average Accuracy: {scores['test_accuracy'].mean()}")
    print(f"Average Precision: {scores['test_precision'].mean()}")
    print(f"Average Recall: {scores['test_recall'].mean()}")
    print(f"Average F1-score: {scores['test_f1'].mean()}")
    print("-" * 20)

Evaluating Logistic Regression
Average Accuracy: 0.837866613172283
Average Precision: 0.7283989238912378
Average Recall: 0.5513909339409377
Average F1-score: 0.6276300959785683
--------------------
Evaluating Gaussian Naive Bayes
Average Accuracy: 0.8013356506083275
Average Precision: 0.6894150775563972
Average Recall: 0.3610802412743451
Average F1-score: 0.47392356370913263
--------------------
Evaluating Linear Discriminant Analysis
Average Accuracy: 0.8249968179643334
Average Precision: 0.7181146453378185
Average Recall: 0.48384984083424587
Average F1-score: 0.5781115097272282
--------------------
Evaluating Quadratic Discriminant Analysis
Average Accuracy: 0.7953429667232171
Average Precision: 0.6898607358459975
Average Recall: 0.3164691275758202
Average F1-score: 0.43386751180636773
--------------------
